In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import csv
import random

# Scrap data

## Scrap data related job from indeed

To limit the scope, focusing on data-related job postings, e.g. data scientist, data analyst, research scientist, business intelligence, and some others. 

Also decreasing the scope by limiting your search to Australia.


In [ ]:
# URL example:
url = 'https://au.indeed.com/jobs? \
    q=data%20scientist%20 \
    %2480%2C000& \
    l=melbourne& \
    start=10& \
    pp=AAoAAAFcPnYSNwAAAAEZ_sq1AQAyMo7IxDIsyedp6BS6'

About the way the URL is structured:
- note “q=” begins the string for the “what” field on the page, separating search terms with “%20” (i.e. searching for “data%20scientist” jobs)
- when specifying salary, it will parse by commas in the salary figure, so the beginning of the salary will be preceded by %24 and then the number before the first comma, it will then be broken by %2C and continue with the rest of the number (i.e. %2420%2C000 = $20,000)
- note “&l=” begins the string for city of interest, separating search terms with “+” if city is more than one word (i.e. “New+York”) ['%20' also can be separator]
- note “&start=” notes the search result where you want to begin (i.e., start by looking at the 10th result)

Set up initial condition:

- Cities
- Job titles
- Features

In [ ]:
cities = ['Sydney', 'Melbourne', 'Brisbane', 'Adelaide', 'Perth', 
          'Gold%20Coast', 'Newcastle', 'Canberra']

jobtitles = ['data%20scientist', 'data%20analyst', 'research%20scientist', 
             'business%20intelligence', 'data%20engineer', 'business%20analyst',
             'big%20data']

columns = ['city', 'job_title', 'job_link', 'company_name', 'location', 'summary', 'salary']

In [ ]:
def scraping(cities, jobtitles, columns, output):
    filename = output + '-output.csv'
    with open(filename,'wb') as resultFile:
        wr = csv.writer(resultFile, dialect='excel')
        wr.writerow(columns)

        for city in cities:
            for job in jobtitles:
                maxResult = 0
                url = 'http://au.indeed.com/jobs?q=' + str(job) + '&l=' + str(city)
                resp = requests.get(url)
                sou = BeautifulSoup(resp.text, 'lxml')
                l = sou.find('div', id='searchCount')
                if l != None:
                    jobCount = l.text.split(' ')[-1]
                    jobCount = int(jobCount.replace(',',''))
                    if jobCount > 1000:
                        jobCount = 1000

                    for start in range(0, jobCount, 10):
                        url = 'http://au.indeed.com/jobs?q=' + str(job) + '&l=' + str(city) + '&start=' + str(start)
                        page = requests.get(url)
                        print url

                        time.sleep(1) 
                        home_url = 'https://au.indeed.com'
                        soup = BeautifulSoup(page.text, 'lxml')

                        for div in soup.find_all(name='div', attrs={'class':'row'}): 
                            job_post = [] 
                            job_post.append(city) 

                            for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
                                job_post.append(a['title'])
                                jobLink = home_url + a['href']
                                job_post.append(jobLink)
                            print jobLink

                            company = div.find_all(name='span', attrs={'class':'company'}) 
                            if len(company) > 0: 
                                for b in company:
                                    try:
                                        job_post.append(b.text.strip()) 
                                    except:
                                        job_post.append(b.text)
                            else: 
                                sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
                                for span in sec_try:
                                    job_post.append(span.text) 

                            c = div.findAll('span', attrs={'class': 'location'}) 
                            for span in c: 
                                job_post.append(span.text)

                            d = div.findAll('span', attrs={'class': 'summary'}) 
                            for span in d:
                                job_post.append(span.text.strip()) 

                            try:
                                job_post.append(div.find('span', attrs={'class':'no-wrap'}).text) 
                            except:
                                try:
                                    div_two = div.find(name='div', attrs={'class':'sjcl'}) 
                                    div_three = div_two.find('div') 
                                    job_post.append(div_three.text.strip())
                                except:
                                    job_post.append('Nothing_found') 

                            try:
                                wr.writerow(job_post)
                            except: #some post has no company name
                                continue

In [ ]:
scraping(cities, jobtitles, columns, 'data_job')

## Scrap developer job

In [ ]:
import csv, requests, time
from bs4 import BeautifulSoup

cities = ['Sydney', 'Melbourne', 'Brisbane', 'Adelaide', 'Perth', 'Gold%20Coast', 'Newcastle', 'Canberra']

columns = ['city', 'job_title', 'job_link', 'company_name', 'location', 'summary', 'salary']

jobtitles2 = ['java', 'python', 'C++', 'ruby', 'php', 'javascript', 'sql', '.net']

scraping(cities, jobtitles2, columns, 'developer_job')

# Preprocess data-related job data

## Add description

In [ ]:
import pandas as pd
import time
jobs = pd.read_csv('output.csv')

df = jobs.drop_duplicates()
df.shape

In [ ]:
df = df[df.salary != 'Nothing_found']
df.shape

In [5]:
def link2descrip(link):
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'lxml')

        job_description = []
        for li in soup.find_all(name='li'):
            job_description.append(li.text.strip())

        print link
        time.sleep(1)
        return job_description
    except:
        return None


In [ ]:
df['description'] = df.job_link.map(link2descrip)

In [ ]:
df.to_csv('output_add_descrip.csv')

## Checking duplication

In [ ]:
fix = pd.read_csv('output_add_descrip.csv')
fix.drop(['Unnamed: 0', 'job_link'], axis=1, inplace=True)
fix.shape

In [ ]:
a = fix.drop_duplicates()
a.reset_index(inplace=True)
a.shape

## Process salary

In [ ]:
a.drop(['summary', 'index'], axis=1, inplace=True)
print a.shape
a = a.dropna()
print a.shape

def salaryCountType(x):
    listsplit = str(x).split(' ')
    return listsplit[-1]

a['salaryCountType'] = a.salary.map(salaryCountType)

In [ ]:
def formatSalary(x):
    listsplit = str(x).split(' ')
    salarytype = listsplit[-1]
    
    num = []
    for e in listsplit:
        try:
            num.append(float(e.replace('$','').replace(',','')))
        except:
            continue
                
    if salarytype == 'year':
        return np.mean(num)
    elif salarytype == 'month':
        return (np.mean(num) * 12)
    elif salarytype == 'week':
        return (np.mean(num) * 52)
    elif salarytype == 'day':
        return (np.mean(num) * 260)
    else: # pay by hour
        return (np.mean(num) * 7.5 * 260)

a['salaryAmount'] = a.salary.map(formatSalary)

a.drop('salary', axis=1, inplace=True)

## Process description

In [ ]:
def formatDes(x):
    strl = x.split('\', u\'')
    for i,s in enumerate(strl):
        if s.startswith('u') or s.startswith('['):
            strl[i] = s[2:]
            
    strl = [x for x in strl if len(x.split(' ')) > 1 and len(x) < 200]
    return strl

a.description = a.description.map(formatDes)
a.head()

In [ ]:
import re
import string

def split2words(x):
    words = []
    for e in x:
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        text = regex.sub('', e)
        words += text.split()
    return words

a.description = a.description.map(split2words)

In [ ]:
a.to_csv('jobs.csv')

## Process for developer job

In [3]:
jobs = pd.read_csv('developer_job-output.csv')
print jobs.shape

df = jobs.drop_duplicates()
print df.shape

df.dropna(inplace=True)
print df.shape

df = df[df.salary != 'Nothing_found']
df.shape

(18731, 7)
(14694, 7)
(14593, 7)


/Users/alex/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(3210, 7)

In [ ]:
df['description'] = df.job_link.map(link2descrip)

df.drop('job_link', axis=1, inplace=True)
print df.shape

df.drop_duplicates(inplace=True)
print df.shape

df.to_csv('developer_output_add_descrip.csv')

https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AZLQvL0eTbPFT73Wkr_fTZ6lAUxbA2JeVsEs5B33cdgms0xSE7SUh7ufOBM5QvjQRjaalDaJSBCCbRy5crR8M5Qx2qAc9AeUUNCb8m9bvdWHguhBUkUMDXSxku3F_MQE5KlTAwnlEkFAm7NumssKHntbZ2vlmNFNPyUNQhqmJbbtx1qD6_CoxLbAeV3CJu4fk0mqrU6uEkzsiqssVTvTMqGr2NSydivXOXtu1AUDRU-h6pC8VSvZhxOblNsj7MQCOEWldl_ozPwp2cKZqwaEALIxzhl4EbDOOFu9nOO0zLLU16Zd1THx39FMi3na89A3TWg886HRymmJi3YmrhHjIf159wJGZvBza6Uo9VBQC3rqANi_nQX2JvgqZ0X7f9dLyFyiDlMvU75KsYiYF7ZX2njI5b6pJW5Oz5BePR-XVe7A==&p=1&sk=&fvj=1
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AZLQvL0eTbPFT73Wkr_fTZ6lAUxbA2JeVsEs5B33cdgms0xSE7SUh7ufOBM5QvjQRjaalDaJSBCCbRy5crR8M5Qx2qAc9AeUUNCb8m9bvdWHguhBUkUMDXSxku3F_MQE5KlTAwnlEkFAm7NumssKHntbZ2vlmNFNOI5mXCKmSea7h8xCJilPIzvGa6XYk8XlX00MbVLw1oRzPQlLuQiPM8OJBdBQyvFCkNvM6vCKJCgoIAanw8SiFyAtFA6VI8JRoeGnL6CpaiyTRcPw8EDpaYf909tn1N2yTKsfIVL9QHipwerB7mb5eOlNoTX-eDs91IbvsDUNONdodD0iCcj2YTycsjyWLOyPuLTyWoSUyTvuW9LCCVX_kR02PEqwg4Cw06uXMM6j_6d4C8heKoDv963c47p2HhHPWDMtNL0ksSZw==&p=7&sk=&fvj=1
https://

https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AZLQvL0eTbPFT73Wkr_fTZ6lAUxbA2JeVsEs5B33cdgms0xSE7SUh7ufOBM5QvjQRjaalDaJSBCCbRy5crR8M5Qx2qAc9AeUUNCb8m9bvdWHguhBUkUMDXSxku3F_MQE5KlTAwnlEkFAm7NumssKHntbZ2vlmNFNPcf6AJ_zq9kg-OKFbmkzEHug0jOcppiUXhyQb6rAkL_H4cIdmsES5_QdTCoVL2cWpkCWedsbZSf0ZUGOzoUrUj-4009WMvxTbQ_shuakfejIgzVt1GtgGdp7OB8zwMlFDgZj8Lhqhp5ggZEXtzq2dNV1CKf-dkc0pkjfuBKy_Hr98Nk71TvL2od9xVVnOHQhPWvrZZhdnjuPOcpzmiByBDzwVEICyP2PYRRPqi3cIL-qexzXrbWxLLvpWW8BUTbxZYySaiEMp-lg==&p=6&sk=&fvj=1
https://au.indeed.com/rc/clk?jk=fbce8e43aa1bc063&fccid=06921d11c91c117f
https://au.indeed.com/rc/clk?jk=72ea40778cd797ff&fccid=86fd58bb45436ba3
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AZLQvL0eTbPFT73Wkr_fTZ6lAUxbA2JeVsEs5B33cdgms0xSE7SUh7ufOBM5QvjQRjaalDaJSBCCbRy5crR8M5Qx2qAc9AeUUNCb8m9bvdWHguhBUkUMDXSxku3F_MQE5KlTAwnlEkFAm7NumssKHntbZ2vlmNFNMl6vu_NOZTlhZi6N8seo2uNnsppf0FdHArzJ16KlnbL6EF1Oyjlm7epUGXS3grB_qzT67wEeqfamPusgYsuhBHuRpF1CXh0rlOdM3mi0OQLn5HIUam5FhkVjcIIuueXtw32wsZf5qv0cG3yCyUDNf

https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AZLQvL0eTbPFT73Wkr_fTZ6lAUxbA2JeVsEs5B33cdgms0xSE7SUh7ufOBM5QvjQRjaalDaJSBCCbRy5crR8M5Qx2qAc9AeUUNCb8m9bvdWHguhBUkUMDXSxku3F_MQE5KlTAwnlEkFAm7NumssKHnfuR3FdDpxdD25GLRyGaCpkYw-cMDBBPyrBaePgB13uC7FkgzdBKEVc2pdbene5MS2YpGj0YyP_HPxE-CbHInq0VT-FBpiZbFv1v9DQJz8yWd087IpDBd93wOZcweHNDSsUHEJvk1fsxCfWQ8CdkxCAVNNSzphmEfKjQiUreQBjhklxF8l3uaLZOKZPS1ICddRRNtVKhi7b-Lstwredza0qLXP8B3jBZi-s7Lidz7nj4rQ44FephcAjmyxIIeLTK5ws7QkBmN_KzgfYMVbLQKmg==&p=2&sk=&fvj=1
https://au.indeed.com/rc/clk?jk=bed8e97384d854a9&fccid=c9831d9e5a2d4cdc
https://au.indeed.com/rc/clk?jk=7e152605d725e865&fccid=087fdafe1fd631b1
https://au.indeed.com/rc/clk?jk=3c08b12f143e2b4c&fccid=c9831d9e5a2d4cdc
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AZLQvL0eTbPFT73Wkr_fTZ6lAUxbA2JeVsEs5B33cdgms0xSE7SUh7ufOBM5QvjQRjaalDaJSBCCbRy5crR8M5Qx2qAc9AeUUNCb8m9bvdWHguhBUkUMDXSxku3F_MQE5KlTAwnlEkFAm7NumssKHnfuR3FdDpxdBHxhV5ncAR-3IFQf_5C7BaF2Nm_ymTk6IUkfVftOFnFlrvhneRrxk3GFSWYUrmrHH28o4Vvr7Qecu

https://au.indeed.com/rc/clk?jk=cf4e30c32071e7c6&fccid=c70fbb69945c84e0
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AZLQvL0eTbPFT73Wkr_fTZ6lAUxbA2JeVsEs5B33cdgms0xSE7SUh7ufOBM5QvjQRjaalDaJSBCCbRy5crR8M5Qx2qAc9AeUUNCb8m9bvdWHguhBUkUMDXSxku3F_MQE5KlTAwnlEkFAm7NumssKHnfuR3FdDpxdCI2LdGw6Hh3thMgeLpt8y7miTvqA3bIbon-4yrTuhAwD6R9y1dyAO-y9OmxBFozXlNAsYltHl8ID2Y24w6JaKSCp60pV_h_6QEkxjE7VJh24bUCFgHMeec4u-Ed2BjkfdsM1rcQ4pS9ciiN9nwm645au8-0MxAYGj_YRKex7B7OTJBp0Qsa0evhwBQbCr3o5SbEps6of5NSUI6kZFaQAuL6rNhqfzb52i87g3VJWRATbOtEAAj5qOtXap3Sq3IoxY43gLNZCeO6w==&p=6&sk=&fvj=1
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AZLQvL0eTbPFT73Wkr_fTZ6lAUxbA2JeVsEs5B33cdgms0xSE7SUh7ufOBM5QvjQRjaalDaJSBCCbRy5crR8M5Qx2qAc9AeUUNCb8m9bvdWHguhBUkUMDXSxku3F_MQE5KlTAwnlEkFAm7NumssKHnfuR3FdDpxdAAC2B3l9MEyvNwyuI57rkD3S3zcJo0hRZSXVopHXz1DRg_jNhRFCpFO8r4n0aw2DQwwedBfsIVfyU1dTXMRD0yxlOy1bIP4vnKDixKZSvZYsTQi3CsY_8cCXN_fCz0mzFhFh55rhQXQEQ630nPGDFMJU1ABLKv4lfQbdFrT_vmbYWgNhY9fmQUfN9im-U8P008WhK99c5im8-U0mpberq0P9qhnL-

https://au.indeed.com/rc/clk?jk=f0533d7f9b6ee270&fccid=77087bd1709a8148
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AZLQvL0eTbPFT73Wkr_fTZ6lAUxbA2JeVsEs5B33cdgms0xSE7SUh7ufOBM5QvjQRjaalDaJSBCCbRy5crR8M5Qx2qAc9AeUUNCb8m9bvdWHguhBUkUMDXSxku3F_MQE5KlTAwnlEkFAm7NumssKHnfuR3FdDpxdBlWLH8sHHOprLCyYjI7bhB3b-xle3KLWWFKdfFXsCLLU77Gncj3QoFrkdLdYqcsEQjM63ZQaGgDLQdi2PvlAFbAthYe8KFWgZR8T31RAxDPCvTCqDOCksTgFJB2CXNczfabrvtFjytiBdGUijlp_dashAal8go9gdR0c3umBX_BBSd5IiprR8UKzgTJpU16vNcPmbYNPUUnHFFYzDdLvZzrvh5ObZ3dKFbQWXmajtsBiefF9paeY4Widi_WUw1w6FMizxN1S4dsw==&p=5&sk=&fvj=1
https://au.indeed.com/rc/clk?jk=6e0ece7f59ff797b&fccid=958b2d6b9afc05bb
https://au.indeed.com/rc/clk?jk=b8106c08d5e8380f&fccid=f63ded28dd6ab0aa
https://au.indeed.com/rc/clk?jk=786fba4a33fbe8ca&fccid=86b33657c5f56739
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AZLQvL0eTbPFT73Wkr_fTZ6lAUxbA2JeVsEs5B33cdgms0xSE7SUh7ufOBM5QvjQRjaalDaJSBCCbRy5crR8M5Qx2qAc9AeUUNCb8m9bvdWHguhBUkUMDXSxku3F_MQE5KlTAwnlEkFAm7NumssKHnfuR3FdDpxdDMJJb

...

https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BBhZYZZ7bY_Q2-fWzXB8nQYCzWpr80zNz3fSsAHgtb9kSWSTCEDVtJC46uuzZ5ixTjDGxB2VpVenB8F7nctnF1WXIB4omisiIT-AU6HKGdIDBGlq9iThKk-k-YzMDAqmif9OS5M7JYLtT2TvpZY2mWte4qTH1kFxVLBrjf0BtOhkyHFnYCWmDGvI1QPUdOhuOQGbIclDbnnnXzXysRqjyQDNAh7zvX0RBmKjbqiYP4kUrHrYau8myagrKkFK_ZWoyoDxC-8zM3EcL2xkT-rNSsecZp96fQ8uA0MA3ZQdTAZir2_vXJS5HmQ8PCqkmsPi_cU0GJX1A3Syn6ScBYqQ5dZiqVc8FLI8aPod_lXyEK1xRMYQZTXgYLu3A-yrqn0gwiKuoYo-JbHcUt9VI8f55oJNIg-7pqyDrxUxXeXGieyUsg9HEli8Z7cSqicCqfT5nLSuznhJvlIHz_iS_1Op0WlVFP96nsriiMsQvvO-CVAIs4BTs1w4Bz7dyIG9aDk3mEus4iIo_Ro3ZDKxefpmsq3gFgzEpw2icP-4zO6kCsDHY_QG3ty5CkWIZ-szLxjSl57vfk6y9LaQ==&p=3&sk=&fvj=0
https://au.indeed.com/rc/clk?jk=c4f7015e7f319311&fccid=0d3a3c9e16872169
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BBhZYZZ7bY_Q2-fWzXB8nQYCzWpr80zNz3fSsAHgtb9kSWSTCEDVtJC46uuzZ5ixTjDGxB2VpVenB8F7nctnF1WXIB4omisiIT-AU6HKGdIDBGlq9iThKk-k-YzMDAqmif9OS5M7JYLhJrF97ghgeBeovEOlm4zquBJ6MVhA8sWh3xR3Dfg3gQhIax-uVF6UqIa1H1kWIT4H-EJV4W-KjgH4c-ke5

https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BBhZYZZ7bY_Q2-fWzXB8nQYCzWpr80zNz3fSsAHgtb9kSWSTCEDVtJC46uuzZ5ixTjDGxB2VpVenB8F7nctnF1WXIB4omisiIT-AU6HKGdIDBGlq9iThKk-k-YzMDAqmif9OS5M7JYLhF96QtUY_riZqhZHTIgW49hfijZw3-ldqONQhXjTQWP4UBTDA8ADr10ef5BOqA4mEx2On0JBW_QLBOOPtfkYS69ai50hs78uQlLX7nqDAOP-l0rLbNP9elAaIz6_Gy3U0ZLXMqfsi-pTT928BGCbKr9MfuFMv2txHFFmFHFXOAzm2jRbBJdpcHkI2OHJmBcTUdwHw-VVde9dzXu1Upx0EkdBPlVYIOw9GEu07ftsC2L9vNm6qS0Ssahyb1gT7t-kQ0gdukDpoSnKvtBJiur0mdwF8hijk-0RpwrKOkXoLwCofRGi8nC2_LKavYSTD8WDgSNEKFkOZ3kCH833bjXSJs_xmUlGS7lovmDj4p18EB8K-X1Gdb7gCkY4Q87ag8SAg8dEp6k9urcdHnC7cDqvDlKLnS1ApifE4cMWSk-c8j0o3Yu2fXuiPgFLsoiB7TKXg==&p=1&sk=&fvj=0
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C0JJUhbQVgzgDGx-dsYjlRJhHO4kZNv2z9s6LVeGBT1zOpjKM400f8vs6t0tYbmeIvHFb0emIq-feFs4GJsa3zrc0wSvK3EX17LvzGdwir2xXFeOsdN6K4PHWmeveDifbADLhLd84sOBmvFtUbGBlM6blwjOZqRGoIKvuiiws3XlGIZad8K6oGVdrbWFQpeJ7gbn9UU7XcPoBeD6gZIXpjMpRHMjR8kbv0k5-olsAAuEr9jI0YBfCBtDpnmKK3v5zWfWm-cgRl3K3K6B9EsT_0pG4zTL39sHMd-oE

https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0A59wcYcY34xyza-tGHHvoFi9VmmQCxgEiiRlhSWHWwxeJmlODOL-n2iN0W3blsGCfOgT_PNkmY89NwZpCPjswSf2V-6OxXz3wXks0pa_qCSEDJbu71buawfvM_xVeB81rdqQ8Iv8HaZ4rQDB5M82yAskKJtKRaACOO-1tY1iuzGumdVDpc_ZUaZ0uI7M-OR7-GLBZ0c0A4i4cpfqclEb1b6Jw7bYsutnnE7P3iLEysv2EEf1MftvA5CJjYRM1OkmhhzRclQA-298DwW5msRLVdWZm8XmMdh7Tsgd26ydIJ8Tmkm4RoLeoVmj8LBjMsTzd4Mhm1VFIIxTagrLbH3nlPJFZnXwC77NqYbVKeqpcVBPUnKq_ML5hTE1AZJe1YCd99yhpDViWOcMyxwqXmqpuMjlve5djtOj8D9uXjRz-DyA==&p=5&sk=&fvj=1
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B-i65Vz1NE2ltaHzTGFhVZlLsj7U06fPnKQvZm_GfHDX4xoIyjarUr3DdUgZQ8EGpVPrtoH0VKnRybN45FED3dgQX9VpQPi76YzHApgeD8QoJQdONcrk884D0C91hsdAzlwAW68K2yvqPBND4BfOojGTIM-MxkoF-Sv682o_eaAbjEcapVNauUbdEYtw4Sbf--t6eYvFkTjP_ReFw4lMYrZUo5r9nQcrbYlHgUqb336v4cvPWS4EwPR4yPrF-XjWZvbYviXOgX2doFtJyMGOVH5kUJE97PNYzqtf0l_t6AGI7FRyZZfm6G-Z-MGqXekeyxrlgxJa1N9CBm-8Xs_7gp1OS3Kq4Ty1CiOnVi1YNEr57BBEcQNmTg58OHILCDmz5Xy1UEoKrOuokvT2yoZDgWntuWamC0QMf0WJv7G7qtwnVtGuyQbx-MCbP6NmVxu8zIr1K

https://au.indeed.com/rc/clk?jk=dd9dbb503a47dff7&fccid=dd616958bd9ddc12
https://au.indeed.com/rc/clk?jk=3f8bfa067211964d&fccid=f6aacd676612b346
https://au.indeed.com/company/Eniax-Pty-Ltd/jobs/Customer-Service-Representative-be4004c237897a71?fccid=ed1b89b8e09adfa4
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BwQOinhhjytFuGzOalMzRs2-0HuZwQVlOp6LTq_X3BOplot1e9n7ywxrqIo5Eo-2gi3RO5YYY5DsHMLeE9KtYH56TzkdJUkbPdQQqb9o20jVxgBb7rJWcKTymd1E14eYwBy1bYT5avmrvsMAFqbRQ7L-a-t0Qy4q-NW5fK-HOgFU4nIaqvtD-Kl_lWZ6t_wjYPjrM5Y_0z8X8HAe5yJ-LnvJs2U2CIzpCJOGyYtrZC5ZuHMPCsb4Nii4GfNoz-hdQg_GltnXZ624XslK_OmeXruhe-UFwAF82n39C-7FiR96izL-Wk3EvV6ZZ49pCjiR8x_-y2AwOV-xaCSQ6mVDPRFTXA6_guTSbfIEtRET4i7y58CkgNbakwCrakAQA3GK2g4d2FoeNaGNSTZPufR6PxWkk3YsypB6Y=&p=6&sk=&fvj=1
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BwQOinhhjytFuGzOalMzRs2-0HuZwQVlOp6LTq_X3BOplot1e9n7ywxrqIo5Eo-2gi3RO5YYY5DsHMLeE9KtYH56TzkdJUkbPdQQqb9o20jVxgBb7rJWcKTymd1E14eYwBy1bYT5avmrvsMAFqbRQ7L-a-t0Qy4q-rBoG8ZvPr97hvLnjo9EWrMzYrLlf2iXG3T6b73Z

https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BwQOinhhjytFuGzOalMzRs2-0HuZwQVlOp6LTq_X3BOplot1e9n7ywxrqIo5Eo-2gi3RO5YYY5DsHMLeE9KtYH56TzkdJUkbPdQQqb9o20jVxgBb7rJWcKTymd1E14eYwBy1bYT5avmrvsMAFqbRQ79lMsKlpFXjq0t7Nxv7QSDsL2X-JoE0_Mk8l6PBBmkh7HI_3u4iWljBJeqzCUh7WXvGPh6VRyRWLZes2MuNY8AtL4nyy2d_J57ajudMVjhrZKQEv-sM7Aq8_8alu8bfyx1ubF_JnsWb6TsDTZawKPZJy4Twv1tJGn76sFyJ6fCEccJGgYijWhT8B1Jj80Td_DpdVGxXT_CtQCQimTPV-RjbOitPKCQQs96dwIVMTojCtBYsf4PlN0cWGGnHozGV-buQtlxyaelbg=&p=5&sk=&fvj=1
https://au.indeed.com/rc/clk?jk=939b57eb4157e892&fccid=ea1f64b071408cc5
https://au.indeed.com/company/A-N-C-LINK-PTY-LTD/jobs/Sale-Consultant-Representative-02588dc9fce03fbf?fccid=9a0f808d168af9ca
https://au.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BwQOinhhjytFuGzOalMzRs2-0HuZwQVlOp6LTq_X3BOplot1e9n7ywxrqIo5Eo-2gi3RO5YYY5DsHMLeE9KtYH56TzkdJUkbPdQQqb9o20jVxgBb7rJWcKTymd1E14eYwBy1bYT5avmrvsMAFqbRQ79lMsKlpFXjqI9FrE0CYQEVLA0MW8qmpV4s3Ucldc2Z7wsPF-5K5IgESPMezUSh8ItTe1AuDmnCU4vt_aFo0e31xqlhRSBC_NRfCIo3aephXVQpPoMYSuOC